In [ ]:
# Instalar googletrans
pip install googletrans==4.0.0-rc1


In [ ]:
from googletrans import Translator
from google.colab import files
import time
import pickle
import os

# Subir el archivo a Colab
uploaded = files.upload()

# Obtener el nombre del archivo subido
file_path = list(uploaded.keys())[0]
file_name, file_extension = os.path.splitext(file_path)

# Leer el contenido del archivo original
with open(file_path, 'r', encoding='utf-8') as file:
    srt_content = file.read()

# Inicializar el traductor
translator = Translator()

# Dividir el contenido en bloques de subtítulos
srt_blocks = srt_content.split('\n\n')

# Cargar progreso previo si existe
progress_file = f'{file_name}_progress.pkl'
try:
    with open(progress_file, 'rb') as pf:
        translated_blocks = pickle.load(pf)
        start_index = len(translated_blocks)
        print(f"Continuando desde el bloque {start_index + 1}")
except FileNotFoundError:
    translated_blocks = []
    start_index = 0

# Función para traducir un bloque de subtítulo
def translate_block(block):
    lines = block.split('\n')
    if len(lines) > 2:
        translated_lines = [lines[0], lines[1]]
        for line in lines[2:]:
            try:
                translated = translator.translate(line, src='en', dest='es')
                if translated and translated.text:
                    translated_lines.append(translated.text)
                else:
                    raise ValueError("Respuesta de traducción inválida o vacía")
            except Exception as e:
                print(f"Error al traducir la línea: {line}\n{e}")
                translated_lines.append(line)  # Mantén la línea original en caso de error
        return '\n'.join(translated_lines)
    else:
        return block

# Traducir cada bloque
for i in range(start_index, len(srt_blocks)):
    try:
        translated_blocks.append(translate_block(srt_blocks[i]))
        # Guardar progreso después de cada bloque
        with open(progress_file, 'wb') as pf:
            pickle.dump(translated_blocks, pf)
        # Pausar para evitar la sobrecarga de la API
        time.sleep(1)
    except Exception as e:
        print(f"Error al traducir el bloque {i + 1}: {srt_blocks[i]}\n{e}")
        translated_blocks.append(srt_blocks[i])

# Combinar los bloques traducidos en una sola cadena
translated_srt_content = '\n\n'.join(translated_blocks)

# Guardar el contenido traducido en un nuevo archivo
translated_file_path = f'{file_name}_Spanish{file_extension}'
with open(translated_file_path, 'w', encoding='utf-8') as file:
    file.write(translated_srt_content)

print(f'Archivo traducido guardado en: {translated_file_path}')
files.download(translated_file_path)

# Eliminar archivo de progreso
os.remove(progress_file)

